In [ ]:
# Implementattion of text classification in python using spacy and logistic regression:
# https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/

import os
import glob

import numpy as np
import pandas as pd

from sklearn.svm import SVC

In [38]:
events = pd.read_csv('../events/group_2_labelled.csv')
events = events.loc[events['Near Miss Event'].notna(), ]

# event_mapper = {val: enum for enum, val in zip(*events.event_id.factorize())}
# file_mapper = {val: enum for enum, val in zip(*events.filename.factorize())}

events.head()

,event_id,filename,sentence_idx,sentence_text,n_trigger_words,trigger_words,event_text,ORE_DEPOSIT,ROCK,MINERAL,STRAT,LOCATION,TIMESCALE,event_label,reviewed,Near Miss Event,Key trigger phrase
0,a081752_anrep2008eraheedy2103_15107355_16,a081752_anrep2008eraheedy2103_15107355.json,16,mineral occurrences and exploration potential ...,1,['potential'],resource search pty ltd. september 2008 2 the ...,['uranium'],"['calcrete', 'granite', 'sediments', 'silts', ...",[],[],"['nabberu basin', 'western australia', 'wester...","['paleoproterozoic', 'cainozoic']",0,True,True,"weakly anomalous, studies to confirm the valid..."
1,a075210_buck_a_ el12_1_2007_11292066_235,a075210_buck_a_ el12_1_2007_11292066.json,235,further drilling in coming years will further ...,1,['further drilling'],of days worked at $ $ construction materials: ...,[],"['coal', 'coal']",[],[],"['ewington', 'premier', 'ewington']",[],0,True,True,prospective areas
2,a075210_buck_a_ el12_1_2007_11292066_246,a075210_buck_a_ el12_1_2007_11292066.json,246,the tenement was applied for on the 12 1 2005 ...,1,['possible'],60 appendix 3: coal resources within el12 1 66...,[],"['coal', 'coal', 'ash', 'coal', 'coal', 'coal']","['diamond', 'sulphur']",[],"['muja', 'collie', 'ewington', 'collie']",[],0,True,True,prospective areas
3,a080379_e80_2574_08atr_12876104_4,a080379_e80_2574_08atr_12876104.json,4,the east kimberley halls creek orogen is widel...,2,"['potential', 'mineralisation']",the planned ground magnetics survey should be ...,['pge'],[],"['gold', 'sulphide', 'sulphide']",[],"['kimberley', 'halls creek orogen', 'australia']",[],0,True,True,NaN
4,a080379_e80_2574_08atr_12876104_5,a080379_e80_2574_08atr_12876104.json,5,hoatson also recognized broad similarities bet...,1,['broad'],the planned ground magnetics survey should be ...,"['pge', 'pge']",[],"['gold', 'sulphide', 'sulphide']",[],"['kimberley', 'halls creek orogen', 'australia']",[],0,True,True,"economic interest, extensively mineralized, ex..."


In [36]:
events.loc[98, 'event_text']

'exploration in the carlow castle area has confirmed the widespread presence of gold and copper mineralisation. the discovery of a blind high grade copper gold zone within low grade copper mineralisation at carlow south, away from the area of main workings, is encouraging. to date, and based on 18 rc holes and five costeans, legend has estimated and published an inferred mineral resource of 1,331,000t at 1.39g t au, 0.56% cu (to 100m depth) for carlow south main lode. subject to a thorough review of the metallurgical testwork conducted to date, a further 20 rc holes (total 1,700 metres) have been recommended to test the carlow south main lode. these holes would bring the drill spacing to 20 metre separation along section lines 40 metres apart, sufficient to estimate an indicated mineral resource under the jorc code. such a mineral resource could be the basis for a type i feasibility study, to determine the broad economics of mining and processing the carlow south main lode. the work by

In [67]:
import string

import spacy
from spacy import displacy

from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin

In [56]:
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_lg')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [52]:
docs = nlp(events.loc[98, 'event_text'])
displacy.render(docs, style = "ent",jupyter = True)

In [68]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
#tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [69]:
from sklearn.model_selection import train_test_split

X = events['event_text']
ylabels = events['Near Miss Event'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [70]:
#end goal:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Logistic Regression Classifier
classifier = LogisticRegression()

# Create pipeline using bag of words(?)
pipe = Pipeline([('vectorizer', bow_vector), ('classifier', classifier)])

# model generation
pipe.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f8ef9f8df70>)),
                ('classifier', LogisticRegression())])

In [78]:
print(f'Class balance for train set:\n{y_train.value_counts()}\n')
print(f'Class balance for test set:\n{y_test.value_counts()}')

Class balance for train set:
1    47
0    23
Name: Near Miss Event, dtype: int64

Class balance for test set:
1    20
0    11
Name: Near Miss Event, dtype: int64


In [71]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.7741935483870968
Logistic Regression Precision: 0.8095238095238095
Logistic Regression Recall: 0.85
